In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 10

In [2]:
LANGUAGES = ['bn', 'en', 'gu', 'hi', 'kn', 'ml', 'mr', 'ne', 'ta', 'te']
LANGUAGE_NAMES = {
    'bn': 'Bengali', 'en': 'English', 'gu': 'Gujarati',
    'hi': 'Hindi', 'kn': 'Kannada', 'ml': 'Malayalam',
    'mr': 'Marathi', 'ne': 'Nepali', 'ta': 'Tamil', 'te': 'Telugu'
}
LENS_TYPES = ['logitlens', 'tunedlens']

BASE_DIR = Path('/mnt/storage/deeksha/indictunedlens/results_with_matches')
OUTPUT_DIR = Path('/mnt/storage/deeksha/indictunedlens/plots_combined')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PLOT_MODE = 'save'   # 'save' or 'show'

In [3]:
def show_or_save(filename=None):
    if PLOT_MODE == 'save' and filename:
        plt.savefig(OUTPUT_DIR / filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ Saved: {filename}")
    elif PLOT_MODE == 'show':
        plt.show()
    else:
        plt.close()


In [4]:
def create_combined_csv(language, lens_type):
    lang_dir = BASE_DIR / f'm_mmlu_{language}'
    if not lang_dir.exists():
        print(f"✗ Missing directory: {lang_dir}")
        return None

    pattern = str(lang_dir / f'{lens_type}_logits_probabilities_*.csv')
    files = glob.glob(pattern)

    if not files:
        print(f"✗ No files found for {language} - {lens_type}")
        return None

    dfs = []
    for f in files:
        try:
            dfs.append(pd.read_csv(f))
        except Exception as e:
            print(f"✗ Error reading {f}: {e}")

    if not dfs:
        return None

    combined_df = pd.concat(dfs, ignore_index=True)
    out_file = lang_dir / f'{lens_type}_combined_logits_probabilities.csv'
    combined_df.to_csv(out_file, index=False, encoding='utf-8')
    print(f"✓ Created combined CSV: {out_file}")

    return combined_df


In [5]:
def ensure_all_combined_csvs():
    print("=" * 80)
    print("ENSURING ALL COMBINED CSV FILES EXIST")
    print("=" * 80)

    for lang in LANGUAGES:
        for lens in LENS_TYPES:
            combined = BASE_DIR / f'm_mmlu_{lang}' / f'{lens}_combined_logits_probabilities.csv'
            if not combined.exists():
                create_combined_csv(lang, lens)

    print("✓ All combined CSVs ensured")
    print("=" * 80)


In [6]:
CSV_CACHE = {}

def preload_all_csvs():
    print("=" * 80)
    print("PRELOADING ALL CSV FILES INTO MEMORY")
    print("=" * 80)

    for lang in LANGUAGES:
        for lens in LENS_TYPES:
            path = BASE_DIR / f'm_mmlu_{lang}' / f'{lens}_combined_logits_probabilities.csv'
            if path.exists():
                CSV_CACHE[(lang, lens)] = pd.read_csv(path)

    print(f"✓ Loaded {len(CSV_CACHE)} DataFrames into memory")
    print("=" * 80)

def get_df(language, lens_type):
    return CSV_CACHE[(language, lens_type)]

# %%
ensure_all_combined_csvs()
preload_all_csvs()

ENSURING ALL COMBINED CSV FILES EXIST
✓ All combined CSVs ensured
PRELOADING ALL CSV FILES INTO MEMORY
✓ Loaded 20 DataFrames into memory


In [7]:
def plot_1_performance_difference_all():
    plt.figure(figsize=(14, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(LANGUAGES)))
    
    for idx, language in enumerate(LANGUAGES):
        df_logit = get_df(language, 'logitlens')
        df_tuned = get_df(language, 'tunedlens')

        differences = []
        layers = []

        for layer in sorted(df_logit['layer'].unique()):
            logit_acc = df_logit[df_logit['layer'] == layer]['match_gold_answer'].mean()
            tuned_acc = df_tuned[df_tuned['layer'] == layer]['match_gold_answer'].mean()
            differences.append(tuned_acc - logit_acc)
            layers.append(layer)

        plt.plot(layers, differences, marker='o', linewidth=2, markersize=5, 
                label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5, linewidth=2)
    plt.xlabel('Layer', fontsize=12)
    plt.ylabel('Accuracy Difference (Tuned − Logit)', fontsize=12)
    plt.title('Performance Difference: Indic-TunedLens − Logit Lens (All Languages)', fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    show_or_save('01_performance_difference_all_languages.png')

In [8]:
def plot_2_layerwise_accuracy_all():
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(LANGUAGES)))
    
    # Logit Lens
    for idx, language in enumerate(LANGUAGES):
        df_logit = get_df(language, 'logitlens')
        logit_acc = df_logit[df_logit['rank'] == 1].groupby('layer')['match_gold_answer'].mean()
        axes[0].plot(logit_acc.index, logit_acc.values, marker='o', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[0].set_title('Logit Lens: Accuracy by Layer', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Layer', fontsize=11)
    axes[0].set_ylabel('Accuracy', fontsize=11)
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[0].grid(alpha=0.3)

    # Tuned Lens
    for idx, language in enumerate(LANGUAGES):
        df_tuned = get_df(language, 'tunedlens')
        tuned_acc = df_tuned[df_tuned['rank'] == 1].groupby('layer')['match_gold_answer'].mean()
        axes[1].plot(tuned_acc.index, tuned_acc.values, marker='s', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[1].set_title('Indic-TunedLens: Accuracy by Layer', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Layer', fontsize=11)
    axes[1].set_ylabel('Accuracy', fontsize=11)
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[1].grid(alpha=0.3)

    plt.suptitle('Layer-wise Accuracy Comparison: All Languages', fontsize=14, fontweight='bold')
    plt.tight_layout()
    show_or_save('02_layerwise_accuracy_all_languages.png')

In [9]:
def plot_3_average_rank_all():
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(LANGUAGES)))
    
    # Logit Lens
    for idx, language in enumerate(LANGUAGES):
        df_logit = get_df(language, 'logitlens')
        logit_rank = df_logit[df_logit['match_gold_answer']].groupby('layer')['rank'].mean()
        axes[0].plot(logit_rank.index, logit_rank.values, marker='o', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[0].invert_yaxis()
    axes[0].set_title('Logit Lens: Average Rank of Correct Answer', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Layer', fontsize=11)
    axes[0].set_ylabel('Average Rank', fontsize=11)
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[0].grid(alpha=0.3)

    # Tuned Lens
    for idx, language in enumerate(LANGUAGES):
        df_tuned = get_df(language, 'tunedlens')
        tuned_rank = df_tuned[df_tuned['match_gold_answer']].groupby('layer')['rank'].mean()
        axes[1].plot(tuned_rank.index, tuned_rank.values, marker='s', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[1].invert_yaxis()
    axes[1].set_title('Indic-TunedLens: Average Rank of Correct Answer', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Layer', fontsize=11)
    axes[1].set_ylabel('Average Rank', fontsize=11)
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[1].grid(alpha=0.3)

    plt.suptitle('Average Rank Comparison: All Languages', fontsize=14, fontweight='bold')
    plt.tight_layout()
    show_or_save('03_average_rank_all_languages.png')

In [10]:
def plot_4_top1_probability_all():
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(LANGUAGES)))
    
    # Logit Lens
    for idx, language in enumerate(LANGUAGES):
        df_logit = get_df(language, 'logitlens')
        logit_prob = df_logit[df_logit['rank'] == 1].groupby('layer')['probability'].mean()
        axes[0].plot(logit_prob.index, logit_prob.values, marker='o', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[0].set_title('Logit Lens: Top-1 Probability', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Layer', fontsize=11)
    axes[0].set_ylabel('Mean Top-1 Probability', fontsize=11)
    axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[0].grid(alpha=0.3)

    # Tuned Lens
    for idx, language in enumerate(LANGUAGES):
        df_tuned = get_df(language, 'tunedlens')
        tuned_prob = df_tuned[df_tuned['rank'] == 1].groupby('layer')['probability'].mean()
        axes[1].plot(tuned_prob.index, tuned_prob.values, marker='s', linewidth=2, 
                    markersize=5, label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    axes[1].set_title('Indic-TunedLens: Top-1 Probability', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Layer', fontsize=11)
    axes[1].set_ylabel('Mean Top-1 Probability', fontsize=11)
    axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    axes[1].grid(alpha=0.3)

    plt.suptitle('Top-1 Probability Comparison: All Languages', fontsize=14, fontweight='bold')
    plt.tight_layout()
    show_or_save('04_top1_probability_all_languages.png')

In [11]:
def plot_5_accuracy_by_position_all():
    plt.figure(figsize=(14, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(LANGUAGES)))
    
    for idx, language in enumerate(LANGUAGES):
        df = get_df(language, 'tunedlens')
        position_acc = df[df['rank'] == 1].groupby('token_position')['match_gold_answer'].mean()
        plt.plot(position_acc.index, position_acc.values, linewidth=2, 
                label=LANGUAGE_NAMES[language], color=colors[idx], alpha=0.8)

    plt.xlabel('Token Position', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Accuracy by Token Position: All Languages (Indic-TunedLens)', fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    show_or_save('05_accuracy_by_token_position_all_languages.png')

In [12]:
def plot_6_final_layer_comparison():
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    final_layer_data = []
    
    for language in LANGUAGES:
        df_logit = get_df(language, 'logitlens')
        df_tuned = get_df(language, 'tunedlens')
        
        final_layer = df_logit['layer'].max()
        
        logit_acc = df_logit[(df_logit['layer'] == final_layer) & (df_logit['rank'] == 1)]['match_gold_answer'].mean()
        tuned_acc = df_tuned[(df_tuned['layer'] == final_layer) & (df_tuned['rank'] == 1)]['match_gold_answer'].mean()
        
        final_layer_data.append({
            'Language': LANGUAGE_NAMES[language],
            'Logit Lens': logit_acc,
            'Indic-TunedLens': tuned_acc,
            'Improvement': tuned_acc - logit_acc
        })
    
    df_final = pd.DataFrame(final_layer_data)
    
    # Accuracy comparison
    x = np.arange(len(LANGUAGES))
    width = 0.35
    
    axes[0].bar(x - width/2, df_final['Logit Lens'], width, label='Logit Lens', alpha=0.8)
    axes[0].bar(x + width/2, df_final['Indic-TunedLens'], width, label='Indic-TunedLens', alpha=0.8)
    axes[0].set_xlabel('Language', fontsize=11)
    axes[0].set_ylabel('Accuracy', fontsize=11)
    axes[0].set_title('Final Layer Accuracy Comparison', fontsize=12, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(df_final['Language'], rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(alpha=0.3, axis='y')
    
    # Improvement
    colors = ['green' if x > 0 else 'red' for x in df_final['Improvement']]
    axes[1].bar(df_final['Language'], df_final['Improvement'], color=colors, alpha=0.7)
    axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[1].set_xlabel('Language', fontsize=11)
    axes[1].set_ylabel('Accuracy Improvement', fontsize=11)
    axes[1].set_title('Improvement: Indic-TunedLens over Logit Lens', fontsize=12, fontweight='bold')
    axes[1].set_xticklabels(df_final['Language'], rotation=45, ha='right')
    axes[1].grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    show_or_save('06_final_layer_comparison.png')

In [13]:
def plot_7_cross_language_summary():
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    summary_data = []
    
    for language in LANGUAGES:
        df_tuned = get_df(language, 'tunedlens')
        
        # Overall accuracy
        overall_acc = df_tuned[df_tuned['rank'] == 1]['match_gold_answer'].mean()
        
        # Average rank of correct answers
        avg_rank = df_tuned[df_tuned['match_gold_answer']]['rank'].mean()
        
        # Top-1 probability
        top1_prob = df_tuned[df_tuned['rank'] == 1]['probability'].mean()
        
        # Accuracy at final layer
        final_layer = df_tuned['layer'].max()
        final_acc = df_tuned[(df_tuned['layer'] == final_layer) & (df_tuned['rank'] == 1)]['match_gold_answer'].mean()
        
        summary_data.append({
            'Language': LANGUAGE_NAMES[language],
            'Overall Accuracy': overall_acc,
            'Avg Rank': avg_rank,
            'Top-1 Probability': top1_prob,
            'Final Layer Accuracy': final_acc
        })
    
    df_summary = pd.DataFrame(summary_data)
    
    # Plot 1: Overall Accuracy
    axes[0, 0].bar(df_summary['Language'], df_summary['Overall Accuracy'], alpha=0.8, color='steelblue')
    axes[0, 0].set_title('Overall Accuracy', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy', fontsize=11)
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(alpha=0.3, axis='y')
    
    # Plot 2: Average Rank
    axes[0, 1].bar(df_summary['Language'], df_summary['Avg Rank'], alpha=0.8, color='coral')
    axes[0, 1].set_title('Average Rank of Correct Answer', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Average Rank', fontsize=11)
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].invert_yaxis()
    axes[0, 1].grid(alpha=0.3, axis='y')
    
    # Plot 3: Top-1 Probability
    axes[1, 0].bar(df_summary['Language'], df_summary['Top-1 Probability'], alpha=0.8, color='mediumseagreen')
    axes[1, 0].set_title('Mean Top-1 Probability', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Probability', fontsize=11)
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(alpha=0.3, axis='y')
    
    # Plot 4: Final Layer Accuracy
    axes[1, 1].bar(df_summary['Language'], df_summary['Final Layer Accuracy'], alpha=0.8, color='orchid')
    axes[1, 1].set_title('Final Layer Accuracy', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('Accuracy', fontsize=11)
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(alpha=0.3, axis='y')
    
    plt.suptitle('Cross-Language Performance Summary (Indic-TunedLens)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    show_or_save('07_cross_language_summary.png')

In [14]:
def plot_8_layerwise_improvement_distribution():
    plt.figure(figsize=(14, 8))
    
    improvement_by_layer = []
    
    for language in LANGUAGES:
        df_logit = get_df(language, 'logitlens')
        df_tuned = get_df(language, 'tunedlens')
        
        for layer in sorted(df_logit['layer'].unique()):
            logit_acc = df_logit[df_logit['layer'] == layer]['match_gold_answer'].mean()
            tuned_acc = df_tuned[df_tuned['layer'] == layer]['match_gold_answer'].mean()
            improvement = tuned_acc - logit_acc
            
            improvement_by_layer.append({
                'Layer': layer,
                'Language': LANGUAGE_NAMES[language],
                'Improvement': improvement
            })
    
    df_imp = pd.DataFrame(improvement_by_layer)
    
    # Create pivot table for heatmap
    heatmap_data = df_imp.pivot(index='Language', columns='Layer', values='Improvement')
    
    sns.heatmap(heatmap_data, cmap='RdYlGn', center=0, annot=False, 
                fmt='.3f', cbar_kws={'label': 'Accuracy Improvement'})
    
    plt.title('Layer-wise Improvement Heatmap: Indic-TunedLens over Logit Lens', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Layer', fontsize=12)
    plt.ylabel('Language', fontsize=12)
    plt.tight_layout()
    show_or_save('08_layerwise_improvement_heatmap.png')

In [15]:
print("\n" + "=" * 80)
print("GENERATING 8 COMBINED PLOTS")
print("=" * 80 + "\n")

plot_1_performance_difference_all()
plot_2_layerwise_accuracy_all()
plot_3_average_rank_all()
plot_4_top1_probability_all()
plot_5_accuracy_by_position_all()
plot_6_final_layer_comparison()
plot_7_cross_language_summary()
plot_8_layerwise_improvement_distribution()

print("\n" + "=" * 80)
print("✓ ALL 8 COMBINED PLOTS GENERATED SUCCESSFULLY")
print("=" * 80)


GENERATING 8 COMBINED PLOTS

✓ Saved: 01_performance_difference_all_languages.png
✓ Saved: 02_layerwise_accuracy_all_languages.png
✓ Saved: 03_average_rank_all_languages.png
✓ Saved: 04_top1_probability_all_languages.png
✓ Saved: 05_accuracy_by_token_position_all_languages.png
✓ Saved: 06_final_layer_comparison.png
✓ Saved: 07_cross_language_summary.png
✓ Saved: 08_layerwise_improvement_heatmap.png

✓ ALL 8 COMBINED PLOTS GENERATED SUCCESSFULLY
